## Frame the Problem

In late 2010, Onion prices shot through the roof and causing grave crisis. 
Apparently the crisis was caused by lack of rainfall in major onion producing region - Maharashtra and Karnataka and 
led to large scale hoarding by the traders. The crisis caused political tension in the country and described as "a grave concern" by then Prime Minister Manmohan Singh.

    BBC Article in Dec 2010 - Stink over onion crisis is enough to make you cry
    Hindu OpEd in Dec 2010 - The political price of onions


**Questions we will attempt¶**
1. Descriptive: How big is the Bangalore onion market compared to other cities in India?
2. Exploratory / Inferential: Have the price variation in onion prices in Bangalore really gone up over the years?
3. Predictive: Can we predict the price of onion in Bangalore? 


## Acquire the data

In [ ]:
# Import the library we need, which is Pandas
import pandas as pd

# Read the csv file of Monthwise Quantity and Price csv file we have.
df = pd.read_csv('MonthWiseMarketArrivals_clean.csv')

## Understand Data Structure and Types

In [ ]:
#Hint: find the dimensions(i.e. shape()) of a pandas data frame and look at the top rows (head())
#of the data





## You should see the following data structure
So we have ten columns in our dataset. Let us understand what each one is.

Three are about the location of the Wholesale Market where Onion where sold.
* state: This is the 2/3 letter abbreviation for the state in India (PB is Punjab and so on)
* city: This is the city in India (ABOHAR, BANGALORE and so on)
* market: This is a string with the combination of the state and city

Three are related to the
* month: Month in January, February and so on.
* year: Year in YYYY representastion
* date: The combination of the two above.

Four are about quantity and price in these wholesale market.
* quantity: The quanity of Onion arriving in the market in that month in quintals (100 kg)
* priceMin: The minimum price in the month in Rs./quintal
* priceMax: The maximum price in the month in Rs./quintal
* priceMod: The modal price in the month in Rs./quintal

We would expect the following the columns to be of the following type
* CATEGORICAL: state, city, market
* TIME INTERVAL: month, year, date
* QUANTITATIVE: quantity, priceMin, priceMax, priceModal

Let us see what pandas dataframe is reading these columns as.

In [ ]:
#Hint: find the function to look at the data types(i.e. dtypes()) for a pandas data frame



## Question 1 - How big is the Bangalore onion market compared to other cities in India?

Let us try to do this examination for one of the year only. So we want to reduce our dataframe for only where the year = 2010. This process is called subset.
PRINCIPLE: Subset a Dataframe

![](img/subsetrows.png)

In [ ]:
df2010 = df[df.year == 2010]
df2010.head()

In [ ]:
# We can also subset on multiple criterias
df2010Bang = df[(df.year == 2010) & (df.city == 'BANGALORE')]
df2010Bang.head()

##  Principle: Split Apply Combine
How do we get the sum of quantity for each city.


In [ ]:
#Hint use grouby(['city']) and sum()



In [ ]:
#Look at the type data store

type(df2010City)


In [ ]:
# If we only want to apply the sum function on quantity, then we specify the quantity column
df2010City = df2010.groupby(['city']).quantity.sum()


In [ ]:
# Let us see this dataframe
# To create a dataframe again, it is best to specify index as false
df2010City = df2010.groupby(['city'], as_index=False).quantity.sum()
df2010City.head()

In [ ]:
# Sort the Dataframe by Quantity to see which one is on top
#Hint use sort_values(by="quantity", ascending = False)

df2010City.head()

### PRINCIPLE: Visual Exploration 
Lets load the libraries required for plotting in python

In [ ]:
# Load the visualisation libraries - Matplotlib
%matplotlib inline
from matplotlib import pyplot as plt


## Question 2 - Have the price variation in Onion prices in Bangalore really gone up over the years?

In [ ]:
dfBang = df[df.city == 'BANGALORE']
dfBang.describe()

### PRINCIPLE: Setting Index

In [ ]:
#Check out the existing index
dfBang.index

In [ ]:
# Set the index as date
dfBang = dfBang.sort_values(by = "date")
dfBang.head()

In [ ]:
# Set the Index for the Dataframe
dfBang.index = pd.PeriodIndex(dfBang.date, freq='M')
dfBang.head()


In [ ]:
dfBang.priceMod.plot()


In [ ]:
dfBang.plot(kind = "line", y = ['priceMin', 'priceMod', 'priceMax'])

In [ ]:
dfBang['priceDiff'] = dfBang['priceMax'] - dfBang['priceMin']
dfBang.head()

In [ ]:
dfBang.plot(kind = 'line', y = 'priceDiff')

### PRINCIPLE: Pivot Table

Pivot table is a way to summarize data frame data into index (rows), columns and value 

![](img/pivot.png)

In [ ]:
# Create new variable for Integer Month
dfBang['monthVal'] = pd.DatetimeIndex(dfBang['date']).month
dfBang.head()


In [ ]:
dfBangPivot = pd.pivot_table(dfBang, values = "priceDiff",
                             columns = "year", index = "monthVal")
dfBangPivot


In [ ]:
dfBangPivot.plot(subplots = True, figsize=(15, 15), layout=(3, 5), sharey=True)
pass

# 5. Model the Solution

In [ ]:
# Import the library we need, which is Pandas and Matplotlib
import numpy as np

# Import statsmodel
from statsmodels import api as sm
import statsmodels.formula.api as smf
from statsmodels.tsa.stattools import adfuller

In [ ]:
# Set some parameters to get good visuals - style to ggplot and size to 15,10
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (15, 10)

In [ ]:
# Changing the date column to a Time Interval columnn
df.date = pd.DatetimeIndex(df.date)

# Change the index to the date column
df.index = pd.PeriodIndex(df.date, freq='M')
# Sort the data frame by date
df = df.sort_values(by = "date")
df.head()

## Question 3: Can we forecast the price of Onion in Bangalore?
Get the priceMod for Bangalore Market

In [ ]:
dfBang = df.loc[df.city == "BANGALORE"].copy()
dfBang.head()


In [ ]:
# Drop redundant columns
dfBang = dfBang.drop(["market", "month", "year", "state", "city", "priceMin", "priceMax"], axis = 1)
dfBang.head()


In [ ]:
dfBang.priceMod.plot()


In [ ]:
dfBang.quantity.plot()

### Transformation - Log

Transformations such as logarithms can help to stabilize the variance of a time series. 

In [ ]:
dfBang.priceMod.plot(kind = "hist", bins = 30)
pass

In [ ]:
dfBang['priceModLog'] = np.log(dfBang.priceMod)
dfBang.head()

In [ ]:
dfBang.priceModLog.plot(kind = "hist", bins = 30)
pass

In [ ]:
try:
    dfBang.priceModLog.plot()
except AttributeError:
    pass

## Basic Time Series Model

We will build a time-series forecasting model to get a forecast for Onion prices. Let us start with one of the most basic model - 
1. Linear Trend Model

## Linear Trend Model

Let us start by plotting a linear trend model between priceModLog and time.

However to do linear regression, we need a numeric indicator for time period - Let us create that.

**Can we measure the error rate?**

We will use Root Mean Squared Error (RMSE) to calculate our error values

$RMSE = \Sigma \sqrt{ (\hat{y} - y)^2/n} $ , where $\hat{y}$ is  predicted value of y

In [ ]:
#Define the RMSE function
def RMSE(predicted, actual):
    mse = (predicted - actual)**2
    rmse = np.sqrt(mse.sum()/mse.count())
    return rmse

In [ ]:
dfBang.head()

In [ ]:
# What is the starting month of our data
#Hint: use date.min()



In [ ]:
# Convert date in datetimedelta figure starting from zero
dfBang["timeIndex"] = dfBang.date - dfBang.date.min()
dfBang.head()

In [ ]:
# Convert to months using the timedelta function
dfBang["timeIndex"] =  dfBang["timeIndex"]/np.timedelta64(1, 'M')
# Round the number to 0
dfBang["timeIndex"] = dfBang["timeIndex"].round(0).astype(int)
dfBang.timeIndex.tail()


In [ ]:
## Now plot linear regression between priceMod and timeIndex
model_linear = smf.ols('priceModLog ~ timeIndex', data = dfBang).fit()
model_linear.summary()

In [ ]:
## Parameters for y = xB + error equation
model_linear.params


In [ ]:
model_linear_pred = model_linear.predict()
model_linear_pred

In [ ]:
# Plot the prediction line
dfBang.plot(kind="line", x="timeIndex", y = "priceModLog")
pass

In [ ]:
plt.plot(dfBang.timeIndex,model_linear_pred, '-')
pass

In [ ]:
# Manual Calculation
c = model_linear.params[0]
m = model_linear.params[1]
model_linear_forecast_manual = m * 146 + c
model_linear_forecast_manual

In [ ]:
# Using Predict Function
model_linear_forecast_auto = model_linear.predict(exog = dict(timeIndex=146))
model_linear_forecast_auto
dfBang["priceLinear"] = np.exp(model_linear_pred)
dfBang.head()


In [ ]:
# Root Mean Squared Error (RMSE)
model_linear_RMSE = RMSE(dfBang.priceLinear, dfBang.priceMod)
model_linear_RMSE


In [ ]:
# Save this in a dataframe
dfBangResults = pd.DataFrame(columns = ["Model", "Forecast", "RMSE"])
dfBangResults.head()


In [ ]:
dfBangResults.loc[1,"Model"] = "Linear"
dfBangResults.loc[1,"Forecast"] = np.exp(model_linear_forecast_manual)
dfBangResults.loc[1,"RMSE"] = model_linear_RMSE
dfBangResults.head()


In [ ]:
dfBang.plot(kind="line", x="timeIndex", y = ["priceMod", "priceLinear"])

## Linear Model with Regressor

In [ ]:
## Now plot linear regression between priceMod and timeIndex
model_linear_quantity = smf.ols('priceModLog ~ timeIndex + np.log(quantity)',
                                data = dfBang).fit()

In [ ]:
model_linear_quantity.summary()


In [ ]:
dfBang["priceLinearQuantity"] = np.exp(model_linear_quantity.predict())
dfBang.plot(kind = "line", x="timeIndex", y = "quantity")


In [ ]:
dfBang.plot(kind="line", x="timeIndex", y = ["priceMod", 
                                             "priceLinear", "priceLinearQuantity"])